<style>div.container { width: 100% }</style>
<img style="float:left;  vertical-align:text-bottom;" height="65" width="172" src="../assets/holoviz-logo-unstacked.svg" />
<div style="float:right; vertical-align:text-bottom;"><h2>HoloViz overview</h2></div>
<br><br>

This notebook is intended to present an overview of HoloViz as a set of slides that you can follow without having to run any code. You can use this document either as a talk, using [RISE](https://github.com/damianavila/RISE), or as a regular Jupyter notebook. Even though you do not *need* to execute any code to follow this talk, you can execute cells (with full user interaction) as long as you have a Python server running for the notebook.

<div style="display: block;"><center>
<img src="../assets/holoviz-logo-stacked.svg" width=170>
<h1>Revealing your data (nearly) effortlessly,<br>at every step in your workflow</h1>
<br>
<img src="../assets/pn_hv_gv_bk_ds_pa.png" width="50%" style="margin: 0px 25%">
<center></div>

# Workflow from data to decision

<div style="clear:left;">
<img src="../assets/workflow.png" width=40% style="margin: 0px 20px; float:left;">
<br>
If there's no visualization at any of these stages, you're flying blind.
<br><br>
But visualization is often skipped as too hard to construct, particularly for big data.
<br><br>
What if it were simple to visualize anything, anywhere?
<br><br><br>
</div>

<div style="clear:left;">
<img src="../assets/landscape_hv_nx.png" width=65% align="left" style="margin: 0px 30px">
<H1>Good news/<br>
Bad news</H1><br>
Lots of choices!
<br>
Too hard to<br>
try them all,<br>
learn them all, or<br>
get them to work together.
<br><br><br>
</div>

<div style="clear:left;">
<img src="../assets/landscape_hv_nx_pyviz.png" width=65% 
align="left" style="margin: 0px 30px">

<H1>HoloViz:</H1>
<br><br>
Seamless interoperability<br>for browser-based<br>viz tools
<br><br>
Supported by Anaconda, Inc.
<br><br><br>
</div>
<div style="clear:left;"></div>

# HoloViz Goals:

- Full functionality in browsers (not desktop)
- Full interactivity (inside and out of plots)
- Focus on Python users, not web programmers
- Start with data, not coding
- Work with data of any size
- Exploit general-purpose SciPy/PyData tools
- Focus on 2D primarily, with some 3D
- Avoid entangling your data, code, and viz:
   * Same viz/analysis code in Jupyter, Python, HPC, ...
   * Widgets/apps in Jupyter, standalone servers, web pages
   * Jupyter as a tool, not part of the results

# Exploring Pandas Dataframes

If your data is in a Pandas dataframe, it's natural to explore it using the ``.plot()`` method (based on Matplotlib).  Let's look at a [dataset of the number of cases of measles and pertussis](http://graphics.wsj.com/infectious-diseases-and-vaccines/#b02g20t20w15) (per 100,000 people) over time in each state:

In [ ]:
import pandas as pd

df = pd.read_csv('../data/diseases.csv.gz')
df.head()

Just calling ``.plot()`` won't give anything meaningful, because it doesn't know what should be plotted against what:

In [ ]:
%matplotlib inline

df.plot();

But with some Pandas operations we can pull out parts of the data that make sense to plot:

In [ ]:
import numpy as np

by_year = df[["Year","measles"]].groupby("Year").aggregate(np.sum)
by_year.plot();

Here it is easy to see that the 1963 introduction of a measles vaccine brought the cases down to negligible levels.

# Exploring Data with hvPlot and Bokeh

The above plots are just static images, but if you import the `hvplot` package, you can use the same plotting API to get fully interactive plots with hover, pan, and zoom in a web browser:

In [ ]:
import hvplot.pandas # noqa: adds hvplot method to pandas objects

by_year.hvplot()

In [ ]:
# Only needed here because we use matplotlib much later on...
import holoviews as hv
hv.extension('bokeh', 'matplotlib', width="100")

Here the interactive features are provided by the [Bokeh](http://bokeh.pydata.org) JavaScript-based plotting library. But what's actually returned by this call is something called a [HoloViews](http://holoviews.org) object, here specifically a HoloViews [Curve](http://holoviews.org/reference/elements/bokeh/Curve.html).  HoloViews objects *display* as a Bokeh plot, but they are actually much richer objects that make it easy to capture your understanding as you explore the data:

In [ ]:
import holoviews as hv
vline = hv.VLine(1963).opts(color='black')

m = by_year.hvplot() * vline * \
    hv.Text(1963, 27000, " Vaccine introduced", halign='left')
m

while still always being able to access the original data involved for further analysis:

In [ ]:
print(m)
m.Curve.I.data.head()

For other plotting libraries, a given visualization that you construct is a dead end -- if you want to change it in some way, you'll need to reconstruct it from scratch with different settings.  

Because HoloViews objects preserve your original data, you can now do *more* with your data than you could before, including anything you could do with the raw data, plus overlaying (as above), laying out in subfigures, slicing, sampling, setting options, and many other operations.

For instance, with HoloViews it's simple to break down the data in different ways.  You can inspect each state individually:

In [ ]:
measles_agg = df.groupby(['Year', 'State'])['measles'].sum()
by_state = measles_agg.hvplot('Year', groupby='State', width=500, dynamic=False)

by_state * vline

Or pull out a couple of those to put side by side:

In [ ]:
by_state["Texas"].relabel('Texas') + by_state["New York"].relabel('New York')

Or to compare four states over time by overlaying:

In [ ]:
states = ['New York', 'New Jersey', 'California', 'Texas']
measles_agg.loc[1930:2005, states].hvplot(by='State') * vline

Or by faceting:

In [ ]:
measles_agg.loc[1930:2005, states].hvplot('Year', col='State', width=200, height=150, rot=90) * vline

Or as a different type of plot, such as a bar chart:

In [ ]:
measles_agg.loc[1980:1990, states].hvplot.bar('Year', by='State', rot=90)

Or with additional information, such as error bars:

In [ ]:
df_error = df.groupby('Year').agg({'measles': [np.mean, np.std]}).xs('measles', axis=1)
df_error.hvplot(y='mean') * hv.ErrorBars(df_error, 'Year').redim.range(mean=(0, None)) * vline

If we really want to invest a lot of time in making a fancy plot, we can customize it to try to show *all* the yearly data about measles at once:

In [ ]:
def nansum(a, **kwargs):
    return np.nan if np.isnan(a).all() else np.nansum(a, **kwargs)

In [ ]:
heatmap = df.hvplot.heatmap('Year', 'State', 'measles', reduce_function=nansum,
    logz=True, height=500, width=900, xaxis=None, flip_yaxis=True, clim=(1, np.nan))

aggregate = hv.Dataset(heatmap).aggregate('Year', np.mean, np.std)
agg = hv.ErrorBars(aggregate) * hv.Curve(aggregate).opts(xrotation=90)
agg = agg.options(height=200, show_title=False)

marker = hv.Text(1963, 800, u'\u2193 Vaccine introduced', halign='left')

In [ ]:
(heatmap + (agg * marker).opts(width=900)).cols(1)

If you prefer, you can choose Matplotlib to render your HoloViews plots, though you give up the interactive pan, zoom, and hover from Bokeh:

In [ ]:
mpl = by_state * hv.VLine(1963).opts(color="black") * \
      hv.Text(1963, 1000, "  Vaccine introduced", halign='left')
hv.output(mpl, backend='matplotlib')

As you can see, these tools make it very quick to explore your data in a browser, and if you choose HoloViews+Bokeh plots, you can have full interactivity with very little code even for quite complex datasets.

# Interactive statistical plots

For high-dimensional datasets with additional data variables, we can compose all the above faceting methods as needed.

For instance, let's look at the Iris dataset:

In [ ]:
from bokeh.sampledata.iris import flowers as iris

iris.tail()

We can now look at all these relationships at once, interactively:<span style="display:block; margin-top:-12px;"> </span>

In [ ]:
hvplot.scatter_matrix(iris, c='species')

# Branching out: large data, geo data, custom controls

HoloViz is a modular suite of tools, and when you need capabilities not handled by Bokeh and HoloViews (and optionally hvPlot) as above, you can bring those in:
 
* [**GeoViews**](http://geoviews.org): Visualizable geographic HoloViews objects
* [**Datashader**](http://datashader.org): Rasterizing huge HoloViews objects to images quickly
* [**Param**](https://param.pyviz.org): Declarative parameters
* [**Panel**](https://panel.pyviz.org): Making it simple to work with widgets inside and outside of a notebook context
* [**Colorcet**](https://colorcet.pyviz.org): perceptually uniform colormaps for big data

We'll look at a large(ish) dataset of 2 million earthquakes on a map, with the following caveat:

<div class="alert alert-warning" role="alert">
  The following examples rely on dynamic updates from a live Python server, and will not update fully when viewed statically on a standard website.
</div>

In [ ]:
import dask.dataframe as dd
import datashader as ds
from colorcet import palette
from holoviews.element.tiles import EsriImagery

topts = hv.opts.Tiles(width=700, height=600, bgcolor='black', 
                      xaxis=None, yaxis=None, show_grid=False)
tiles = EsriImagery().opts(topts) 
earthquakes  = dd.read_parquet('../data/earthquakes.parq', engine='fastparquet').persist()
colormaps = {n: palette[n] for n in ['fire','bgy','bgyw','bmy','gray','kbc']}

**NOTE**: If you didn't use conda to install holoviz, you might get an error message about missing SNAPPY. You can get all the dependencies by installing python-snappy using conda: `conda install python-snappy` or follow [these steps](https://github.com/andrix/python-snappy#dependencies).

In [ ]:
x, y = ds.utils.lnglat_to_meters(earthquakes.longitude, earthquakes.latitude)
projected_earthquakes = earthquakes.assign(x=x, y=y).persist()

In [ ]:
import hvplot.dask # noqa: adds hvplot method to dask objects

def view(cmap=colormaps['fire'], alpha=1, reverse_colormap=False):
    cmap = cmap if not reverse_colormap else cmap[::-1]
    return tiles.opts(alpha=alpha) * projected_earthquakes.hvplot.points(
        'x', 'y', datashade=True, cmap=cmap
    )

view()

As you can see, you can specify geo plots easily and if your HoloViews objects are too big to visualize in a browser directly, you can add `datashade()` to render them into images dynamically on zooming, etc.

**NOTE:** HoloViews includes support for basic web-based map tiles as used here, but if you need to work flexibly with different geographic projections, you'll want to install [GeoViews](https://geoviews.org) as well. See the notebook on [Geographic Data](../tutorial/07_Geographic_Data.ipynb) for more information.

You can also easily add widgets to control filtering, selection, and other options interactively, either here in the notebook or by putting the same code in a separate file and running it as a standalone server:

In [ ]:
import panel as pn
explorer = pn.interact(view, cmap=colormaps, alpha=(0, 1.), reverse_colormap=False)

pn.Row(pn.Column('# Earthquake Explorer', explorer[0]), explorer[1]).servable()

Here we used the Panel `interact` function to create a simple app based on the `view` function, and then we mixed and matched some of its components to lay it out in rows and columns as you see above.  

In this simple app, the `view` function is called whenever *any* of the parameters change (alpha, colormap, or location), triggering a full rerender, but you can get a more responsive interface if you take the time to declare which computations depend on which parameters (see the [Deploying Bokeh Apps tutorial](../tutorial/13_Deploying_Bokeh_Apps.ipynb)).  

Either way, the app should work the same here in the notebook (if you have a live Python process) or as a standalone server by calling `panel serve` with either the name of a Python file with the above code or simply the name of this notebook (where it will run the notebook code and serve any objects marked `.servable()`).)

As you can see, the HoloViz tools let you integrate visualization into everything you do, using a small amount of code that reveals your data's properties and captures your understanding of it. The rest of these tutorials will break down each of the topics covered above, showing you step by step how to work with your own data using these tools.

Thanks to all of the HoloViz contributors!